In [1]:
import pandas as pd


In [2]:
path ='/home/milo/PCIC/Maestría/2doSemestre/seminario/github/data/climate_data_calibration'

# Cargamos los países y continentes
continentes = pd.read_csv("{}/continents-according-to-our-world-in-data.csv".format(path))
continentes_set = list(set(continentes["Continent"]))
continentes_set.remove("Antarctica")

# Generamos un diccionario con los países que pertenecen a cada continente
continentes.loc[continentes["Continent"]=="North America","Continent"] = "America"
continentes.loc[continentes["Continent"]=="South America","Continent"] = "America"

continentes_set = list(set(continentes["Continent"]))
continentes_set.remove("Antarctica")
continentes_dict = {}
for cont in continentes_set:
    continentes_dict[cont] = list(continentes.query("Continent =='{}'".format(cont)).Entity)

# Agregamos países al diccionario de continentes
continentes_dict["America"] = continentes_dict["America"] + ["Bahamas, The"]
continentes_dict["Europe"] = continentes_dict["Europe"] + ["Czech Republic","Former Czechoslovakia","Former Serbia and Montenegro","Former Yugoslavia","Germany, East","Germany, West","Former U.S.S.R."]
continentes_dict["Africa"] = continentes_dict["Africa"] + ["Palestinian Territories","Congo (Brazzaville)","Congo (Kinshasa)","Cote dIvoire (IvoryCoast)","Gambia, The","Sudan and South Sudan"]
continentes_dict["Asia"] = continentes_dict["Asia"] + ["Burma (Myanmar)","Korea, North","Korea, South","Macau","Timor-Leste (East Timor)"]

def busca_coincidencia(pais,pais2cont):
    flag = False
    for k,v in pais2cont.items():
        if pais in v:
            flag = True
            continente = k
    
    if flag:
        if (continente=="Europe" or continente=="Africa"):
            continente = "Eurafrica"
        elif (continente=="Asia" or continente=="Oceania"):
            continente = "Asia-Oceania"
        else:
            continente = "America"
            
        return continente
    else:
        return "na"
    

In [3]:
# Cargamos los datos de consumo de petróleo, carbón y gas natural
dict_consumo_energia = {}
dict_consumo_energia['petroleo'] = pd.read_excel("{}/EnergyConsumptionbySector.xlsx".format(path), sheet_name="ConsumptionPetroleum",
                        index_col=None, na_values=['0'],header =0, skiprows=3, nrows=237,usecols = "A:AI")
dict_consumo_energia['carbon'] = pd.read_excel("{}/EnergyConsumptionbySector.xlsx".format(path), sheet_name="ConsumptionNatGas",
                        index_col=None, na_values=['0'],header =0, skiprows=3, nrows=237,usecols = "A:AI")
dict_consumo_energia['gas'] = pd.read_excel("{}/EnergyConsumptionbySector.xlsx".format(path), sheet_name="ConsumptionCoal",
                        index_col=None, na_values=['-'],header =0, skiprows=3, nrows=237,usecols = "A:AI")
dict_consumo_energia['renovable'] = pd.read_excel("{}/EnergyConsumptionbySector.xlsx".format(path), sheet_name="ConsumptionRenewables",
                        index_col=None, na_values=['0'],header =0, skiprows=3, nrows=237,usecols = "A:AI")


def get_annual_data(datos,etiqueta,pais2cont):
    
    datos.drop(columns='Unnamed: 1', inplace=True)
    datos.rename(columns={ x:("country" if y==1979 else y) for x,y in zip(datos.columns,range(1979,2013))},inplace=True)
    
    datos["region"] = datos["country"].apply(lambda x: busca_coincidencia(x,pais2cont))
    datos.query("region !='na'",inplace=True)

    datos = pd.melt(datos.drop(columns='country'), id_vars='region', value_vars=[y for y in range(1980,2013)]).rename(columns={'region':'region','variable':'year','value':'value'})
    datos["energia"] = etiqueta
    
    datos.loc[datos["value"]=="--","value"] = "0"
    datos["value"] = datos["value"].apply(lambda x: float(x))
    return datos[["year","region","energia","value"]]

In [4]:
df_energia_fosil = pd.DataFrame()

for energia in ["petroleo","gas","carbon"]:
    data_energia = get_annual_data(dict_consumo_energia[energia],energia,continentes_dict)
    df_energia_fosil = pd.concat([df_energia_fosil,data_energia])
    
df_energia_fosil = df_energia_fosil.groupby(["year","region","energia"], as_index=False).sum()
df_energia_fosil_total = df_energia_fosil.groupby(["year","region"], as_index=False).sum()
df_energia_fosil_total["energia"] ="fosil"

df_energia_fosil_total = pd.concat([df_energia_fosil_total[["year","region","energia","value"]],df_energia_fosil])
df_energia_fosil_total.to_csv("{}/energia_fosil_total_regiones.csv".format(path),index=False)

df_energia_renovable_total = get_annual_data(dict_consumo_energia["renovable"],"renovable",continentes_dict)
df_energia_renovable_total = df_energia_renovable_total.groupby(["year","region","energia"], as_index=False).sum()
df_energia_renovable_total.to_csv("{}/energia_renovable_total_regiones.csv".format(path),index=False)


In [5]:

def get_annual_data_advanced(datos,etiqueta,pais2cont,region,lideres):
    
    datos["region"] = datos["country"].apply(lambda x: busca_coincidencia(x,pais2cont))
    datos.query("region !='na'",inplace=True)

    datos = pd.melt(datos, id_vars=['region','country'], value_vars=[y for y in range(1980,2013)]).rename(columns={'region':'region','variable':'year','value':'value'})
    datos["energia"] = etiqueta
    
    datos.loc[datos["value"]=="--","value"] = "0"
    datos["value"] = datos["value"].apply(lambda x: float(x))
    
    consulta = ''

    for k,i in enumerate(lideres):
        consulta += "country == '{}' ".format(i)
        if k != len(lideres)-1:
            consulta += " or "

    advanced_datos = datos.query(consulta)[["year","region","energia","value"]]
    advanced_datos = advanced_datos.groupby(["year","region","energia"], as_index=False).sum()
    advanced_datos["economia"] = "advanced"

    continente_sub = datos.query("region == '{}'".format(region))

    emerging_countries = list(set(continente_sub.country) - set(lideres))

    consulta = ''

    for k,i in enumerate(emerging_countries):
        consulta += "country == '{}' ".format(i)
        if k != len(emerging_countries)-1:
            consulta += " or "
    
    emerging_datos = continente_sub.query(consulta)[["year","region","energia","value"]]
    emerging_datos = emerging_datos.groupby(["year","region","energia"], as_index=False).sum()
    emerging_datos["economia"] = "emerging"

    return pd.concat([advanced_datos,emerging_datos])

In [6]:
# Generamos los datos para las economías líderes de cada región

df_energia_lideres_conti = pd.DataFrame()

lideres_region_dict ={"America":['United States', 'Canada'],
                      "Asia-Oceania":['China','Japan'],
                      "Eurafrica": ['Austria','Belgium','Bulgaria','Croatia','Cyprus','Czech Republic','Denmark','Estonia','Finland',
 'France','Germany','Greece','Hungary','Ireland','Italy','Latvia','Lithuania','Luxembourg','Malta',
 'Netherlands','Poland','Portugal','Romania','Slovakia','Slovenia','Spain','Sweden',"Russia",'Former U.S.S.R.',
   'Former Yugoslavia','Former Serbia and Montenegro','Former Czechoslovakia']}

energias = ['carbon', 'gas', 'petroleo','renovable']

for energia in energias:
    for k,v in lideres_region_dict.items():
        parcial_df = get_annual_data_advanced(dict_consumo_energia[energia],energia,continentes_dict,k,v)
        df_energia_lideres_conti = pd.concat([df_energia_lideres_conti,parcial_df])

df_energia_lideres_conti.to_csv("{}/energia_lideres_regiones.csv".format(path),index=False)

In [7]:
df_energia_lideres_conti_fosil = df_energia_lideres_conti.query("energia !='renovable'")
df_energia_lideres_conti_fosil.to_csv("{}/energia_fosil_desagregado_lideres_regiones.csv".format(path),index=False)
df_energia_lideres_conti_fosil = df_energia_lideres_conti_fosil[['year','region','economia','value']].groupby(['year','region','economia'], as_index=False).sum()

In [8]:
df_energia_lideres_conti_fosil.to_csv("{}/energia_fosil_total_lideres_regiones.csv".format(path),index=False)

In [9]:
df_energia_lideres_conti_reno = df_energia_lideres_conti.query("energia =='renovable'")
df_energia_lideres_conti_reno.to_csv("{}/energia_renovable_total_lideres_regiones.csv".format(path),index=False)

In [10]:
df_energia_lideres_conti_reno

,year,region,energia,value,economia
0,1980,America,renovable,5.57754,advanced
1,1981,America,renovable,5.59067,advanced
2,1982,America,renovable,6.00849,advanced
3,1983,America,renovable,6.38661,advanced
4,1984,America,renovable,6.45758,advanced
...,...,...,...,...,...
28,2008,Eurafrica,renovable,3.43413,emerging
29,2009,Eurafrica,renovable,3.38704,emerging
30,2010,Eurafrica,renovable,3.50218,emerging
31,2011,Eurafrica,renovable,3.45777,emerging


In [11]:
df_energia_lideres_conti_fosil

,year,region,economia,value
0,1980,America,advanced,76.79202
1,1980,America,emerging,12.66992
2,1980,Asia-Oceania,advanced,30.48723
3,1980,Asia-Oceania,emerging,21.62765
4,1980,Eurafrica,advanced,83.75837
...,...,...,...,...
193,2012,America,emerging,28.10399
194,2012,Asia-Oceania,advanced,114.40142
195,2012,Asia-Oceania,emerging,112.85119
196,2012,Eurafrica,advanced,76.69640


In [12]:
df_energia_lideres_conti

,year,region,energia,value,economia
0,1980,America,carbon,22.15037,advanced
1,1981,America,carbon,21.61148,advanced
2,1982,America,carbon,20.25839,advanced
3,1983,America,carbon,19.12118,advanced
4,1984,America,carbon,20.44938,advanced
...,...,...,...,...,...
28,2008,Eurafrica,renovable,3.43413,emerging
29,2009,Eurafrica,renovable,3.38704,emerging
30,2010,Eurafrica,renovable,3.50218,emerging
31,2011,Eurafrica,renovable,3.45777,emerging


In [13]:
df_energia_lideres_conti

,year,region,energia,value,economia
0,1980,America,carbon,22.15037,advanced
1,1981,America,carbon,21.61148,advanced
2,1982,America,carbon,20.25839,advanced
3,1983,America,carbon,19.12118,advanced
4,1984,America,carbon,20.44938,advanced
...,...,...,...,...,...
28,2008,Eurafrica,renovable,3.43413,emerging
29,2009,Eurafrica,renovable,3.38704,emerging
30,2010,Eurafrica,renovable,3.50218,emerging
31,2011,Eurafrica,renovable,3.45777,emerging
